In [2]:
from collections import deque
import numpy as np
import pandas as pd
from data_wrangling import df_full
from pathlib import Path


In [3]:
def distance_calc(lat1, lon1, lat2, lon2):
    earth_radius = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Compute differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula for distance calculation
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Earth radius in kilometers
    earth_radius = 6371.0
    
    # Calculate the distance
    distance = earth_radius * c
    
    return np.round(distance, 2)


In [4]:
def create_matrix(df, airports):
    num_airports = len(airports)

    # Create an adjacency matrix with zeros
    adjacency_matrix = np.zeros((num_airports, num_airports), dtype=float)

    # Fill the adjacency matrix based on the flights
    for index, row in df.iterrows():
        source_index = airports.index(row["Source Airport"])
        dest_index = airports.index(row["Destination Airport"])
        lat1, lon1 = row["Src Latitude"], row["Src Longitude"]
        lat2, lon2 = row["Dest Latitude"], row["Dest Longitude"]
        
        distance = distance_calc(lat1, lon1, lat2, lon2)
        adjacency_matrix[source_index][dest_index] = distance

    # source = [airports.index('JFK'), airports.index('LGA')]
    # sink = [airports.index('SFO')]

    return adjacency_matrix#, source, sink


In [6]:
# df = df_full[df_full["Src City"].str.contains(source) |
#                                                 df_full["Dest City"].str.contains(sink)]
airports = sorted(set(df_full["Source Airport"]).union(set(df_full["Destination Airport"])))
adjacency_matrix, sources, sinks = create_matrix(df_full, airports)

adjacency_df = pd.DataFrame(adjacency_matrix, columns=airports, index=airports)

# ford_fulkerson(adjacency_matrix, sources, sinks)

csv_file_path = "/Users/yuhanburgess/Documents/GitHub/AGP2/csv_files/distance_matrix.csv"
adjacency_df.to_csv(csv_file_path, index=True, header=True) 

